In [19]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, KFold
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

# Load and split data

In [47]:
# sklearn data to pandas dataframe
# X, y = load_wine(return_X_y=True)
data = load_wine()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = pd.Series(data.target)
print(df.shape)
df.head()

(178, 14)


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


In [57]:
### Random training/test data split ###
train_idx, test_idx = train_test_split(df.index.tolist(), test_size=0.2, random_state=42)

X_train = []
y_train = []
X_test = []
y_test = []

for idx, row in df.iterrows():
    if idx in train_idx:
        X_train.append(df.iloc[idx][df.columns != 'target'].tolist())
        y_train.append(int(df.iloc[idx]['target']))
    else:
        X_test.append(df.iloc[idx][df.columns != 'target'].tolist())
        y_test.append(int(df.iloc[idx]['target']))

print('----DATA SET----')
print(f'Train \t | \t Test'.expandtabs(0))
print(f'{len(X_train)} \t | \t {len(X_test)}'.expandtabs(2))

----DATA SET----
Train  |  Test
142    |   36


# Train model

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import RFE

model = RandomForestClassifier()
cv_results = cross_validate(model, X_train, y_train, scoring='roc_auc_ovr', cv=10)
print(f"CV AUC w/o FS: {cv_results['test_score'].mean():.4}")


rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select=3, step=1)
cv_results = cross_validate(rfe, X_train, y_train, scoring='roc_auc_ovr', cv=10)
print(f"CV AUC w. FS: {cv_results['test_score'].mean():.4}")

In [88]:
model = RandomForestClassifier()
cv_folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

i = 1
auc = []
acc = []
mcc = []
best_auc = 0.0
best_acc = 0.0
best_mcc = 0.0
for train_index, val_index in cv_folds.split(X_train, y_train):
    print(f'{i}. CV iteration', end =' '); i += 1

    train_x, valid_x = X_train[train_index], X_train[val_index]
    train_y, valid_y = y_train[train_index], y_train[val_index]

    model.fit(train_x, train_y)

    pred_proba = model.predict_proba(valid_x)
    pred_labels = model.predict(valid_x)

    auc.append(metrics.roc_auc_score(valid_y, pred_proba, multi_class='ovr'))
    acc.append(metrics.accuracy_score(valid_y, pred_labels))
    mcc.append(metrics.matthews_corrcoef(valid_y, pred_labels))
    print(f'Current AUC: {auc[-1]:.4f}, Current Acc: {acc[-1]:.4f}, Current MCC: {mcc[-1]:.4f}')

    if auc[-1] > best_auc:
        best_auc = auc[-1]
        best_model = model

    if acc[-1] > best_acc:
        best_acc = acc[-1]
    
    if mcc[-1] > best_mcc:
        best_mcc = mcc[-1]

print(f'\nMean AUC: {np.mean(auc):.4f} +/- {np.std(auc, ddof=1):.4f}  |  Best AUC: {best_auc:.4f}')
print(f'Mean Acc: {np.mean(acc):.4f} +/- {np.std(acc, ddof=1):.4f}  |  Best Acc: {best_acc:.4f}')
print(f'Mean MCC: {np.mean(mcc):.4f} +/- {np.std(mcc, ddof=1):.4f}  |  Best MCC: {best_mcc:.4f}')

1. CV iteration Current AUC: 1.0000, Current Acc: 1.0000, Current MCC: 1.0000
2. CV iteration Current AUC: 0.9965, Current Acc: 0.9655, Current MCC: 0.9495
3. CV iteration Current AUC: 0.9982, Current Acc: 0.9643, Current MCC: 0.9481
4. CV iteration Current AUC: 1.0000, Current Acc: 1.0000, Current MCC: 1.0000
5. CV iteration Current AUC: 1.0000, Current Acc: 1.0000, Current MCC: 1.0000

Mean AUC: 0.9989 +/- 0.0016  |  Best AUC: 1.0000
Mean Acc: 0.9860 +/- 0.0192  |  Best Acc: 1.0000
Mean MCC: 0.9795 +/- 0.0281  |  Best MCC: 1.0000


# Final testing (only allowed in the end of the Hackathon)

In [ ]:
print(f'----------------Testing classifier ----------------')
X_test_pca = X_test
Y_pred_proba = best_model.predict_proba(X_test_pca)
print('Pred. AUC:', metrics.roc_auc_score(y_test, Y_pred_proba, multi_class='ovr'))
Y_pred = best_model.predict(X_test_pca)
print('Pred. Acc:', metrics.accuracy_score(y_test, Y_pred))
print('Pred. MCC:', metrics.matthews_corrcoef(y_test, Y_pred))
print('Confusion matrix:', metrics.confusion_matrix(y_test, Y_pred).ravel())
print('Total sites:', len(y_test))